In [198]:
import numpy as np
import scipy.signal as signal
import plotly.express as px
import pandas as pd # pour les dataframes
import sounddevice as sd # pour jouer les sons associés aux signaux
import pandas as pd

## 1 Fenêtrage temporel
1. Décrire le fonctionnement de la fonction np.fft.fftshift() , et son intérêt pour l’analyse spectrale. Dé-
terminer la transformation à appliquer sur l’axe fréquentiel pour afficher de manière cohérente le spectre obtenu
après l’application de la fonction np.fft.fftshift() .
--> Elle permet de mettre les BF au centre

In [199]:
help(np.fft.fftshift)

Help on _ArrayFunctionDispatcher in module numpy.fft:

fftshift(x, axes=None)
    Shift the zero-frequency component to the center of the spectrum.

    This function swaps half-spaces for all axes listed (defaults to all).
    Note that ``y[0]`` is the Nyquist component only if ``len(x)`` is even.

    Parameters
    ----------
    x : array_like
        Input array.
    axes : int or shape tuple, optional
        Axes over which to shift.  Default is None, which shifts all axes.

    Returns
    -------
    y : ndarray
        The shifted array.

    See Also
    --------
    ifftshift : The inverse of `fftshift`.

    Examples
    --------
    >>> import numpy as np
    >>> freqs = np.fft.fftfreq(10, 0.1)
    >>> freqs
    array([ 0.,  1.,  2., ..., -3., -2., -1.])
    >>> np.fft.fftshift(freqs)
    array([-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.])

    Shift the zero-frequency component only along the second axis:

    >>> freqs = np.fft.fftfreq(9, d=1./9).reshape(3, 3)
   

2. Grâce aux fonctions signal.windows.hann() et signal.windows.boxcar() , créer deux sé-
quences wH et wB composées de K = 32 points correspondant respectivement aux fenêtres de Hann et rectan-
gulaire.
Afficher ces deux fenêtres en fonction du temps.

In [200]:
K = 32
t = np.linspace(0,K,  32) #len(t) = 32

wH = signal.windows.hann(K)
wB = signal.windows.boxcar(K)

#On affiche les 2 sinaux
fig1 = px.line(x=t, y=wH, title='sequence wH', labels={'x':'t', 'y':'fx'}).show()
fig1 = px.line(x=t, y=wB, title='sequence wB', labels={'x':'t', 'y':'fx'}).show()

3. Calculer la valeur moyenne des 2 fenêtres précédentes ( np.mean() ). En déduire G, le facteur d’amplitude à
appliquer sur wH pour que les deux fenêtres aient la même moyenne

In [201]:
print("apli moyene de wH", np.mean(wH))
print("apli moyene de wB", np.mean(wB))

FAmpli = 1/np.mean(wH)
print("Amplification nécéssaire : ", FAmpli)

apli moyene de wH 0.484375
apli moyene de wB 1.0
Amplification nécéssaire :  2.064516129032258


4. Affichez sur un même graphe en fonction des fréquences réduites, les spectres centrés des deux fenêtres de
même moyenne calculés sur N = 4K points. Retrouver à partir de ces tracés les caractéristiques de chacune
de ces deux fenêtres (cf. TD Signaux continus Ex. 10). En déduire leurs avantages/inconvénients pour l’analyse
spectrale (amplitude des rebonds et résolution fréquentielle)

In [202]:
K = 32
t = np.linspace(1,K, K) #len(t) = 32

wH = signal.windows.hann(K)
wB = signal.windows.boxcar(K)

df = pd.DataFrame({'t': t, 'wB': wB, 'wH à l echelle': wH * FAmpli})
fig = px.line(df, x='t', y=['wB', 'wH à l echelle'],
              labels={'t': 'temps', 'value': 'amplitude', 'variable': 'fenêtre'},
              title='Fenêtres : rectangulaire et Hann (même moyenne)')
fig.show()

- Fenêtre rectangulaire
Lobe principal étroit → bonne résolution fréquentielle
Rebonds (lobes secondaires) très élevés (≈ -13 dB)
- Fenêtre de Hann
Lobe principal plus large → résolution plus médiocre
Rebonds très faibles (≈ -31 dB) 
La fenêtre de Hann atténue les artéfacts mais diminue la capacité à distinguer des fréquences très proches.
La fenêtre rectangulaire donne une meilleure séparation des fréquences, mais avec beaucoup plus de parasites dans le spectre

In [203]:
#fenetres en fréquencielles
N = 4*K
f = (np.arange(N)/N)-0.5

fft_wH = np.fft.fftshift(np.fft.fft(wH, N))
fft_wB = np.fft.fftshift(np.fft.fft(wB, N))

fig = px.line(x=f,
              y=[abs(fft_wB), abs(fft_wH)],
              labels={'x': 'Fréquence (Hz)', 'value': 'Amplitude', 'variable': 'fenêtre'},
              title='Spectres centrés : rectangulaire (wB) et Hann (wH)')
fig.show()

5. Créer maintenant la séquence suivante :
- sB [k] = A1 cos(2πν1t) + A2 cos(2πν2t)
- avec νe = 1 kHz, A1 = 2, ν1 = 82.35 Hz, A2 = 0.1 ν2 = 100.15 Hz, et K = 256 points.
- Puis créer la séquence fenêtrée avec Hann :
- sH [k] = sB [k] · G · wH [k]

In [204]:
A1, A2 = 2, 0.1
nu1, nu2, nue = 82.35, 100.15, 1000
G = FAmpli
K = 256
N = 4*K
k = np.arange(0,N)
t = k/nue
#t = np.linspace(1,N, N)

sB = A1*np.cos(2*np.pi*nu1*t) + A2*np.cos(2*np.pi*nu2*t)
wH = signal.windows.hann(N)
sH = sB*G*wH
fig1 = px.line(x=t, y=sH, title='sequence sH', labels={'x':'t', 'y':'fx'}).show()

fig2 = px.line(x=t, y=sB, title='sequence sB', labels={'x':'t', 'y':'fx'}).show()

6. Afficher sur un même graphe les deux signaux temporels en fonction du temps, puis sur un autre graphe les
spectres centrés de sH [k] et sB [k]. Notez le nombre de pics d’amplitude pour chaque courbe, et relevez les
fréquences auxquelles ces pics apparaissent. Commentez

In [205]:
#spectre de sH
fft_sH = np.fft.fftshift(np.fft.fft(sH, N))
freq = (np.arange(N)/N-0.5)*nue

fig2 = px.line(x=freq, y=abs(fft_sH), 
               title='Spectre centré de sH : fenetre de Hann', 
               labels={'x': 'Fréquence (Hz)', 'y': 'Amplitude'}).show()

#spectre de sB
fft_sB = np.fft.fftshift(np.fft.fft(sB, N))
fig3 = px.line(x=freq, y=abs(fft_sB), 
               title='Spectre centré de sB : fenetre rectangulaire', 
               labels={'x': 'Fréquence (Hz)', 'y': 'Amplitude'}).show()

7. Refaire la même étude avec les paramètres suivants :
- νe = 1 kHz, A1 = 2, ν1 = 88.65 Hz, A2 = 2 ν2 = 100.15 Hz, et K = 64 points. 
- Notez le nombre de pics d’amplitude pour chaque courbe, et relevez les fréquences auxquelles ces pics apparaissent. Commentez

In [206]:
A1, A2 = 2, 2
nu1, nu2, nue = 88.65, 100.15, 1000
G = FAmpli
K = 64
N = 4*K
k = np.arange(0,N)
t = k/nue
#t = np.linspace(1,N, N)

sB = A1*np.cos(2*np.pi*nu1*t) + A2*np.cos(2*np.pi*nu2*t)
wH = signal.windows.hann(N)
sH = sB*G*wH

#spectre de sH
fft_sH = np.fft.fftshift(np.fft.fft(sH, N))
freq = (np.arange(N)/N-0.5)*nue

fig2 = px.line(x=freq, y=abs(fft_sH), 
               title='Spectre centré de sH: fenetre de Hann', 
               labels={'x': 'Fréquence (Hz)', 'y': 'Amplitude'}).show()

#spectre de sB
fft_sB = np.fft.fftshift(np.fft.fft(sB, N))
fig3 = px.line(x=freq, y=abs(fft_sB), 
               title='Spectre centré de sB : fenetre rectangula', 
               labels={'x': 'Fréquence (Hz)', 'y': 'Amplitude'}).show()

## 2 Analyse temps-fréquence
Nous nous intéresserons dans cette partie à l’analyse temps-fréquence d’un signal non stationnaire, i.e. dont le contenu
fréquentiel varie dans le temps. Nous utiliserons pour cela la classe ShortTimeFFT de la librairie scipy.signal.
# 2.1 Création du signal
La séquence étudiée dans cette partie sera composée de 3 séquences mises bout à bout :
- Une séquence monochromatique, d’expression :
s1(t) = cos(2πν1t)
- Une séquence polychromatique, d’expression :
s2(t) = cos(2πν2t) + cos(2πν3t)
- Une séquence contenant un « chirp linéaire », d’expression :
s3(t) = cos(2πν4t2)
1. Créer les 3 séquences s1,s2 et s3 avec les paramètres suivants :
• Durée de chaque séquence : 1 s
• Fréquence d’échantillonnage : νe = 1 kHz
• Fréquences des signaux : ν1 = 100 Hz, ν2 = 50 Hz, ν3 = 200 Hz et ν4 = 200 Hz

In [214]:
# création du signal non stationnaire : 3 segments de 1 s chacun, νe = 1 kHz
nue = 1000  # Hz
K = 1000
N = 3 * K  # nombre total d'échantillons

# fréquences demandées
nu1 = 100.0
nu2 = 50.0
nu3 = 200.0
nu4 = 200.0

t = np.arange(N) / nue

# séquences
s1 = np.cos(2 * np.pi * nu1 * t)                          
s2 = np.cos(2 * np.pi * nu2 * t) + np.cos(2 * np.pi * nu3 * t) 
s3 = np.cos(2 * np.pi * nu4 * (t ** 2))                   

2. Grâce à la commande s = np.concatenate((s1,s2,s3)) , créer la séquence finale et afficher-la en
fonction du temps.

In [208]:
# concaténation en un seul signal et vecteur temps global
s = np.concatenate([s1, s2, s3])
t = np.arange(len(s)) / nue
figS = px.line(x=t, y=(s), 
               title='sequence S', 
               labels={'x': 'Fréquence (Hz)', 'y': 'Amplitude'}).show()


## 2.2 Spectrogramme
1. Tracer le spectre centré de la séquence s en fonction des fréquences réelles et analyser son contenu. Commenter.

In [ ]:
#spectre de sH
fft_sH = np.fft.fftshift(np.fft.fft(s, N))
freq = (np.arange(N)/N-0.5)*nue

fig2 = px.line(x=freq, y=abs(fft_sH), 
               title='Spectre centré de S', 
               labels={'x': 'Fréquence (Hz)', 'y': 'Amplitude'}).show()


2. Rappeler le principe de construction d’un spectrogramme, et les caractéristiques du signal qu’il permet de visualiser.

3. À partir de l’aide de la classe ShortTimeFFT (cliquer ici), identifier la signification des paramètres d’entrée
suivants : win, hop, fs, et mfft.

- win : fenêtre temporelle glissante
- hop : incrément de temps entre deux fenêtres successives
- fs : fréquence d'échantillonnage : relation avec l'intervalle d'echantillonage Ts = 1/fs
- mfft : nombre de points pour le calcul de la FFT si aucun, on utilise la taille du signal win

4. À partir maintenant de la méthode extent et des attributs delta_t, et f, déterminer comment obtenir les
vecteurs temporel (t_spec) et fréquentiel (nu_spec) associés au spectrogramme.

In [ ]:
nu_spec = 1/delta_t

5. Créer deux fenêtres (porte wB, et Hann wH), sur K = 64 points grâce aux fonctions vues dans la partie précédente.
Créer ensuite une instance de la classe ShortTimeFFT , puis calculer le spectrogramme grâce aux commandes
suivantes :

In [211]:
# Paramétrisation des 2 spectrogrammes (1 par fenêtre)
SFT_B = signal.ShortTimeFFT(win=wB, hop=K, fs=nue, mfft=K)
SFT_H = signal.ShortTimeFFT(win=wH, hop=K, fs=nue, mfft=K)
# Calcul des spectrogrammes de s
spectrogram_B = SFT_B.stft(s)
spectrogram_H = SFT_H.stft(s)

ValueError: Attribute mfft=64 needs to be at least the window length m_num=256!

Afficher les 2 spectrogrammes obtenus en fonction du temps et des fréquences à partir des vecteurs déterminés à
la question 4, et la commande suivante :

In [ ]:
# Affichage d'un spectrogramme
px.imshow(np.abs(spectrogram_B), x = t_spec, y = nu_spec,
origin='lower', aspect='auto', labels={'x': 'Temps (s)', 'y':
'Fréquence (Hz)', 'color': 'Magnitude'}, title='Spectrogramme de
s').show()